# Notebook notes

This notebook is for testing training

# Import libs

**setup autoreload**

In [1]:
%load_ext autoreload
%autoreload 2

**configure system path**

In [2]:
import os
import sys

# list directories where packages are stored
# note that the parent directory of te repo is added automatically
GYM_FOLDER = "gym-t99"

# get this notebook's current working directory
nb_cwd = os.getcwd()
# get name of its parent directory
nb_parent = os.path.dirname(nb_cwd)
# add packages to path
sys.path.insert(len(sys.path), nb_parent)
sys.path.insert(len(sys.path), os.path.join(nb_parent, GYM_FOLDER))

**import custom packages**

Use this code if you have not yet loaded gym_t99

In [3]:
import gym
registered = gym.envs.registration.registry.env_specs.copy()

import gym_t99
import t_net

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


**import custom packages**

Use this code if you have already loaded gym_t99, but now you introduced changes and want to re-load it again

In [4]:
import imp

# this code removes environment from gym's registry
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env not in registered:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]

imp.reload(gym_t99)

Remove basic-v0 from registry
Remove t99-v0 from registry


<module 'gym_t99' from '/Users/ianconceicao/239AS_T99_RL_Agent/gym-t99/gym_t99/__init__.py'>

**import out-of-the-box packages**

In [5]:
# import matplotlib
import matplotlib as plt
# configure matplotlib
%matplotlib inline

In [6]:
import numpy as np
from collections import OrderedDict

In [7]:
# import torch
import torch
from torch import nn

# configure torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Workspace

In [8]:
custom_gym = gym.make('gym_t99:t99-v0', num_players = 2, enemy="rand")

In [17]:
frame = custom_gym.render(mode="human")

In [19]:
print(frame[0])

[[10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  3  3  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  3  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  3  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10  0  0  0  0  0  0  0  0  0  0 10 10]
 [10 10 10